In [1]:
!gdown --id 1SBKsb9hy9vmOm5Cfi0GDOrpm-_kqXOyS

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1SBKsb9hy9vmOm5Cfi0GDOrpm-_kqXOyS
To: /content/modi_output.txt
100% 221M/221M [00:04<00:00, 55.0MB/s]


In [2]:
data = open('./modi_output.txt', 'r', encoding='utf-8')
lines = data.readlines()
passages = []
summaries = []

for line in lines:
    line = line.strip()
    if len(line.split('\t')) < 2:
        print(line)
        break
    passages.append('<s>' + line.split('\t')[0] + '</s>') # kobart 학습시에는 bos, eos삽입이 필요
    summaries.append('<s>' + line.split('\t')[1] + '</s>')

data.close()

In [3]:
print("passages:", passages[0])
print("summaries:", summaries[0])

passages: <s>자신의 생각을 불명확하게 표현하는 사람들은 생각도 불명확할 가능성이 높다. 따라서 생각을 명료하게 정리하는 것이 명확한 글쓰기의 시작이다. 존 케네스 갈브레이드(John Kenneth Galbraith) 교수는 “아무리 복잡하고 어려운 주제라고 명쾌한 언어로 설명이 가능하다. 하지만 필자가 그 주제에 대해 완전히 이해하지 못하면 절대로 명확하게 쓸 수 없다.”고 했다. 명료한 글은 명료한 생각에서 나온다. 그렇지만 글을 고치다보면 생각도 명료하게 된다. 명료하게 쓰려면 내용을 단순화해야 한다. 많은 것을 전달하려는 욕심을 버리고 한두가지로 선택하고 거기 집중해야 한다. 앞에서 이야기한 간결성은 명확성에 도움이 된다. 명료함은 복잡한 설득기법을 사용하는 것보다 더 중요하다. 판사는 이해할 수 없는 사실관계 진술로는 설득되지 않는다. 설득기법이 명료함을 해치는 때에는 그 기법을 사용하지 않는다.</s>
summaries: <s>주제가 아무리 복잡하고 어려울 지라도 필자가 주제에 대해 완벽하게 이해하고 있다면 명확한 글쓰기는 당연 가능하다. 명확한 글쓰기를 할 때 가장 중요한 점은 내용의 단순화 및 간결성이고 많은 것을 전달하려는 욕심을 버리는 것이다.</s>


In [4]:
print(len(passages))

160002


## 2. make dataset

huggingface에서 데이터를 쉽게 다루기 위해 datasets으로 묶어줍니다.

In [5]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 28.0 MB/s 
     |████████████████████████████████| 120 kB 71.1 MB/s 
     |████████████████████████████████| 6.6 MB 41.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 22.6 MB/s 
     |████████████████████████████████| 115 kB 76.4 MB/s 
     |████████████████████████████████| 212 kB 68.7 MB/s 
     |████████████████████████████████| 127 kB 58.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_test, y_train, y_test = train_test_split(
    passages[0:1000], # 전부 학습하면 시간이 오려걸려서(1epoch에 1시간 30분정도..) 1000개만 학습
    summaries[0:1000],
    test_size = 0.2,
    shuffle=True
)

In [8]:
print(len(x_train))
print(len(x_test))

800
200


In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
dataset = DatasetDict(
    {
        'train' : Dataset.from_dict({'passage':x_train, 'summary': y_train}),
        'valid' : Dataset.from_dict({'passage':x_test, 'summary':y_test}),
    }
)

In [ ]:
print(dataset['train'][0])

{'passage': '<s>2014년은 독일과 유럽 역사의 최대 사건인 1차 세계대전, 2차 세계대전, 베를린 장벽 붕괴가 100주년, 75주년, 25주년을 맞이한 해였다. 이에 따라 주 교육부 장관회는 학교가 민주주의를 학습하고 경험하는 공간이 되어야 할 것 이라며, 올바른 역사교육을 위한 “기억의 문화(Erinnerungskultur)” 확립을 모든 주정부가 추진해야 할 교육 정책의 중점으로 삼았다. 이를 위해 각 주 교육부는 교과과정에 독일 근대사 교육을 강화하는 한편 학생들이 보다 실질적으로 역사를 경험하도록 전쟁 유적지나 유태인 강제수용소 견학 등 현장학습을 확대하고 전쟁기념일 관련 다양한 프로젝트를 실시하였다. 주 교육부 장관회는 현재 독일 역사에 대해 증언할 당시 세대 증언자들이 감소하고, 이주배경 이 있는 청소년들이 증가하여 이들의 성공적 사회통합을 위한 올바른 독일 역사교육의 필요성이 증가하고 있음을 강 조하였으며, 청소년들이 역사를 비판적, 사실적으로 학습하여 올바른 역사관을 확립할 수 있도록 역사교육을 지원하 고 있다.</s>', 'summary': "<s>2014년은 1차, 2차 세계대전 및 베를린 장벽 붕괴가 각각 100, 75, 25주년을 맞이한 해다. 이에 주 교육부 장관회는 올바른 역사교육을 위한 '기억의 문화'확립을 교육 정책의 중점으로 삼고, 청소년들이 올바른 역사관을 확립할 수 있도록 역사교육을 지원하고 있다.</s>"}


## 3. tokenizing datasets

먼저 kobart-base-v2의 tokenizer를 다운받습니다.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gogamza/kobart-base-v2")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Downloading:   0%|          | 0.00/682k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
print(tokenizer.special_tokens_map)

{'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'}


In [ ]:
sents = ["이순신은 조선 중기의 무신이다.", "이순신 짱"]

In [ ]:
tokenized_sents = tokenizer(sents)

In [ ]:
for keys in tokenized_sents.keys():
    print(keys, ":", tokenized_sents[keys]) # 길이가 다릅니다.

input_ids : [[22577, 19293, 15015, 14059, 15525, 14113, 11467, 16247], [22577, 11467, 1700, 12371]]
token_type_ids : [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0]]
attention_mask : [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]]


tokenizing 옵션을 줄 수 있다!

In [ ]:
tokenized_sents = tokenizer(sents, max_length=6, truncation=True, padding=True)
for keys in tokenized_sents.keys():
    print(keys, ":", tokenized_sents[keys])

input_ids : [[22577, 19293, 15015, 14059, 15525, 14113], [22577, 11467, 1700, 12371, 3, 3]]
token_type_ids : [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
attention_mask : [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 0, 0]]


전체 데이터셋을 tokenizing하여 저장하겠습니다!

In [ ]:
max_input_length = 512
max_target_length = 512

def preprocess_function(examples):
    inputs = [doc for doc in examples["passage"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True) # 단, padding은 주지 않는다.
    # padding은 data_collator가 만들어 줄거다!

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocessed_dataset_sample = preprocess_function(dataset['train'][0:1])

for keys in preprocessed_dataset_sample.keys():
    print(keys, ':', preprocessed_dataset_sample[keys])

input_ids : [[0, 17424, 12005, 15604, 9120, 15251, 22730, 14809, 14690, 12037, 16869, 14370, 19579, 243, 16411, 14370, 19579, 243, 27348, 14137, 10868, 20717, 8981, 14854, 17294, 243, 21016, 17294, 243, 14951, 21135, 19389, 13590, 14088, 20029, 14599, 14302, 14053, 28818, 15212, 15835, 29217, 18528, 10443, 16935, 14058, 15211, 14049, 26028, 24549, 14207, 14033, 19370, 14795, 21841, 15361, 20964, 14353, 14128, 9264, 11764, 12024, 14685, 18296, 313, 14885, 309, 14879, 27276, 314, 306, 20348, 315, 19512, 240, 1547, 22632, 12007, 14537, 14053, 20490, 14634, 14368, 14207, 14468, 25390, 14059, 16439, 14312, 18332, 14650, 14185, 14319, 14053, 14468, 14697, 17263, 9120, 15767, 15604, 19158, 11207, 18580, 24694, 14602, 18631, 15936, 27594, 19348, 15211, 15542, 15755, 14078, 12124, 29456, 14078, 13145, 12037, 18085, 11372, 11908, 11319, 15327, 13587, 14048, 16092, 20140, 12007, 26795, 15755, 19649, 12041, 14342, 14641, 26876, 15043, 27332, 14053, 28818, 15212, 15835, 14475, 15604, 29574, 14225, 

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3543: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


In [ ]:
print(tokenizer.decode(preprocessed_dataset_sample['input_ids'][0]))

<s> 2014년은 독일과 유럽 역사의 최대 사건인 1차 세계대전, 2차 세계대전, 베를린 장벽 붕괴가 100주년, 75주년, 25주년을 맞이한 해였다. 이에 따라 주 교육부 장관회는 학교가 민주주의를 학습하고 경험하는 공간이 되어야 할 것 이라며, 올바른 역사교육을 위한 “기억의 문화(Erinnerungskultur)” 확립을 모든 주정부가 추진해야 할 교육 정책의 중점으로 삼았다. 이를 위해 각 주 교육부는 교과과정에 독일 근대사 교육을 강화하는 한편 학생들이 보다 실질적으로 역사를 경험하도록 전쟁 유적지나 유태인 강제수용소 견학 등 현장학습을 확대하고 전쟁기념일 관련 다양한 프로젝트를 실시하였다. 주 교육부 장관회는 현재 독일 역사에 대해 증언할 당시 세대 증언자들이 감소하고, 이주배경 이 있는 청소년들이 증가하여 이들의 성공적 사회통합을 위한 올바른 독일 역사교육의 필요성이 증가하고 있음을 강 조하였으며, 청소년들이 역사를 비판적, 사실적으로 학습하여 올바른 역사관을 확립할 수 있도록 역사교육을 지원하 고 있다.</s>


In [ ]:
print(tokenizer.decode(preprocessed_dataset_sample['labels'][0]))

<s> 2014년은 1차, 2차 세계대전 및 베를린 장벽 붕괴가 각각 100, 75, 25주년을 맞이한 해다. 이에 주 교육부 장관회는 올바른 역사교육을 위한 '기억의 문화'확립을 교육 정책의 중점으로 삼고, 청소년들이 올바른 역사관을 확립할 수 있도록 역사교육을 지원하고 있다.</s>


동작을 확인했으니, 학습 데이터를 tokenizing해서 저장하겠습니다.

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3543: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
for keys in tokenized_datasets['train'][0].keys():
    print(keys, ':', tokenized_datasets['train'][0][keys])

passage : <s>2014년은 독일과 유럽 역사의 최대 사건인 1차 세계대전, 2차 세계대전, 베를린 장벽 붕괴가 100주년, 75주년, 25주년을 맞이한 해였다. 이에 따라 주 교육부 장관회는 학교가 민주주의를 학습하고 경험하는 공간이 되어야 할 것 이라며, 올바른 역사교육을 위한 “기억의 문화(Erinnerungskultur)” 확립을 모든 주정부가 추진해야 할 교육 정책의 중점으로 삼았다. 이를 위해 각 주 교육부는 교과과정에 독일 근대사 교육을 강화하는 한편 학생들이 보다 실질적으로 역사를 경험하도록 전쟁 유적지나 유태인 강제수용소 견학 등 현장학습을 확대하고 전쟁기념일 관련 다양한 프로젝트를 실시하였다. 주 교육부 장관회는 현재 독일 역사에 대해 증언할 당시 세대 증언자들이 감소하고, 이주배경 이 있는 청소년들이 증가하여 이들의 성공적 사회통합을 위한 올바른 독일 역사교육의 필요성이 증가하고 있음을 강 조하였으며, 청소년들이 역사를 비판적, 사실적으로 학습하여 올바른 역사관을 확립할 수 있도록 역사교육을 지원하 고 있다.</s>
summary : <s>2014년은 1차, 2차 세계대전 및 베를린 장벽 붕괴가 각각 100, 75, 25주년을 맞이한 해다. 이에 주 교육부 장관회는 올바른 역사교육을 위한 '기억의 문화'확립을 교육 정책의 중점으로 삼고, 청소년들이 올바른 역사관을 확립할 수 있도록 역사교육을 지원하고 있다.</s>
input_ids : [0, 17424, 12005, 15604, 9120, 15251, 22730, 14809, 14690, 12037, 16869, 14370, 19579, 243, 16411, 14370, 19579, 243, 27348, 14137, 10868, 20717, 8981, 14854, 17294, 243, 21016, 17294, 243, 14951, 21135, 19389, 13590, 14088, 20029, 14599, 14302, 14053, 28818, 15212, 15835, 29217, 18

## 4. Fine-tuning using KoBART

이제 본격적으로 kobart 모델을 불러와 학습을 해보겠습니다.

In [ ]:
from transformers import BartForConditionalGeneration # generation에서 classification모델 하나 더 부착!

In [ ]:
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v2')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

모델 구조를 확인해봅니다.

In [ ]:
model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [ ]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [ ]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    output_dir = 'model_output',
    evaluation_strategy = 'steps',
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size= batch_size,
    save_total_limit = 2,
    num_train_epochs = 2,
    predict_with_generate = True,
    save_steps = 1000,
    eval_steps = 500
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
# input과 output에 모두 padding을 해준다.

이제 evaluation을 위한 metric을 넣어보겠습니다

In [ ]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=c2ae56201e2102982e24700c1225e7adeebb6e1a7624c54716107d83573b30b3
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [ ]:
from datasets import load_metric

metric = load_metric("rouge")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
!pip install kss==2.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import kss
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence kss로 split한다음 비교하고 있다!
    decoded_preds = ["\n".join(kss.split_sentences(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(kss.split_sentences(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions = decoded_preds, references=decoded_labels, use_stemmer=True)

    # Extract a few results
    result = {key: value.mid.fmeasure*100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred !=tokenizer.pad_tokenizer.pad_token_id) for pred in predictions]
    result['gen_len'] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['valid'],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, passage, token_type_ids. If summary, passage, token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 800
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 300
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than us

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=300, training_loss=1.7391282145182292, metrics={'train_runtime': 114.158, 'train_samples_per_second': 21.023, 'train_steps_per_second': 2.628, 'total_flos': 416335675392000.0, 'train_loss': 1.7391282145182292, 'epoch': 3.0})

In [ ]:
trainer.save_model()

Saving model checkpoint to tmp_trainer
Configuration saved in tmp_trainer/config.json
Model weights saved in tmp_trainer/pytorch_model.bin
tokenizer config file saved in tmp_trainer/tokenizer_config.json
Special tokens file saved in tmp_trainer/special_tokens_map.json


In [ ]:
!cp /content/model_output/pytorch_model.bin /content/drive/MyDrive/
!cp /content/model_output/config.json /content/drive/MyDrive/

cp: cannot stat '/content/model_output/pytorch_model.bin': No such file or directory
cp: cannot stat '/content/model_output/config.json': No such file or directory


## 5. inference

이제 학습된 모델로 inference를 날려보겠습니다

In [ ]:
from transformers import AutoTokenizer, BartForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained('gogamza/kobart-base-v2')
model = BartForConditionalGeneration.from_pretrained('MrBananaHuman/kobart-base-v2-summarization').to('cuda')

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gogamza--kobart-base-v2/snapshots/d9a1f640896cef8dcfd693b1bc57510a2b09a18f/config.json
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Model config BartConfig {
  "_name_or_path": "gogamza/kobart-base-v2",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm

Downloading:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--MrBananaHuman--kobart-base-v2-summarization/snapshots/3e77b6a58c5e4632dad736eb83b86519edfea593/config.json
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Model config BartConfig {
  "_name_or_path": "gogamza/kobart-base-v2",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads

Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--MrBananaHuman--kobart-base-v2-summarization/snapshots/3e77b6a58c5e4632dad736eb83b86519edfea593/pytorch_model.bin
All model checkpoint weights were used when initializing BartForConditionalGeneration.

All the weights of BartForConditionalGeneration were initialized from the model checkpoint at MrBananaHuman/kobart-base-v2-summarization.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BartForConditionalGeneration for predictions without further training.


https://colab.research.google.com/drive/1yUGVmQ0nj8Hd3h0YV6PemQx0FtzpefGB?usp=sharing#scrollTo=XbzZ_IVTtoQe

In [ ]:
import torch

def summarization(text):
    tokenized_text = tokenizer(text, return_tensors='pt', truncation=True).to('cuda')
    with torch.no_grad():
        outputs = model.generate(
            tokenized_text['input_ids'],
            do_sample=True,
            eos_token_id = tokenizer.eos_token_id,
            max_length = 512,
            top_p = 0.7,
            top_k = 20,
            num_beams = 20,
            num_return_sequences = 1,
            no_repeat_ngram_size = 2,
            early_stopping = True
        )
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
summarization("""<s> 로스트아크는 스마일게이트 RPG가 개발한 쿼터뷰 액션 MMORPG다.
2014년 11월 12일 최초 공개했으며 2018년 11월 7일부터 오픈 베타 테스트를 진행하였으며 2019년 12월 4일 정식 오픈했다.
이후 러시아, 일본, 글로벌 서버 순으로 런칭하였다.
플랫폼은 PC 하나뿐이지만 모바일과 콘솔 등 각종 플래솜으로도 개발을 진행하고 있다는 기사가 존재한다. </s>""")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


'로스트아크는 스마일게이트 RPG가 개발한 쿼터뷰 액션 MMORPG로 플랫폼은 PC 하나뿐이지만 각종 플래솜으로도 개발을 진행하고 있다.'